# 🤖 1. Import thư viện

In [1]:
# Import libraries
import os
import torch
import evaluate
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import Any, Dict, List

from datasets import load_dataset, Audio, concatenate_datasets
from transformers import (
    WhisperProcessor,
    WhisperForConditionalGeneration,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    EarlyStoppingCallback
)
from peft import get_peft_model, LoraConfig, TaskType

In [2]:
from huggingface_hub import notebook_login
notebook_login()

MODEL_ID = "openai/whisper-large-v3"
OUTPUT_DIR = "./models/whisper_de_finetune"
CACHE_DIR = os.path.expanduser("~/.cache/huggingface")

In [5]:
from datasets import load_dataset, Audio
from transformers import WhisperForConditionalGeneration, WhisperProcessor

# Load dataset
common_voice = load_dataset(
    "mozilla-foundation/common_voice_13_0",
    "de",
    cache_dir=CACHE_DIR,
    trust_remote_code=True,   # ✅ thêm dòng này
    split={
        "train": "train[:10%]",
        "validation": "validation[:10%]",
        "test": "test[:10%]"
    }
)

# Resample audio
for split in ["train", "validation", "test"]:
    common_voice[split] = common_voice[split].cast_column("audio", Audio(sampling_rate=16000))

# Load model + processor
processor = WhisperProcessor.from_pretrained(MODEL_ID, language="de", task="transcribe", cache_dir=CACHE_DIR)
model = WhisperForConditionalGeneration.from_pretrained(MODEL_ID, cache_dir=CACHE_DIR)
model.freeze_encoder()   # khóa encoder để giảm số tham số cần train

n_shards.json:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

audio/de/train/de_train_0.tar:   0%|          | 0.00/1.72G [00:00<?, ?B/s]

audio/de/train/de_train_1.tar:   0%|          | 0.00/1.66G [00:00<?, ?B/s]

audio/de/train/de_train_2.tar:   0%|          | 0.00/1.65G [00:00<?, ?B/s]

audio/de/train/de_train_3.tar:   0%|          | 0.00/1.58G [00:00<?, ?B/s]

...

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]